# Chapter 7 - RegEx

In [13]:
!powershell [regex]::new ^| Get-Member



   TypeName: System.Management.Automation.PSMethod

Name                MemberType Definition                                                                  
----                ---------- ----------                                                                  
Copy                Method     System.Management.Automation.PSMemberInfo Copy()                            
Equals              Method     bool Equals(System.Object obj)                                              
GetHashCode         Method     int GetHashCode()                                                           
GetType             Method     type GetType()                                                              
Invoke              Method     System.Object Invoke(Params System.Object[] arguments)                      
ToString            Method     string ToString()                                                           
IsInstance          Property   bool IsInstance {get;}                             

In [14]:
import re

In [21]:
phone_re = re.compile(r'\d{3}-\d{3}-\d{4}')
result = phone_re.search('my number is 212-123-4567')
print(f'Found phone number {result.group()}')

Found phone number 212-123-4567


In [16]:
# https://kevinmarquette.github.io/2017-07-31-Powershell-regex-regular-expression/#select-string

In [25]:
!powershell \
$PhoneRe = [regex]::new('\d{3}-\d{3}-\d{4}') ;\
$Result = $PhoneRe.Match('my number is 212-123-4567') ;\
Write-Host "Found phone number $Result.Value"

Found phone number 212-123-4567


### Grouping with parentheses

In [54]:
phone_re = re.compile(r'(\d\d\d)-(\d\d\d-\d\d\d\d)')
match = phone_re.search('my number is 212-123-4567')
print(match)
print(match.group(1))
print(match.group(2))
print(match.group(0))
print(match.group())
area_code, number = match.groups()
print(f'Area: {area_code} Number: {number}')

<re.Match object; span=(13, 25), match='212-123-4567'>
212
123-4567
212-123-4567
212-123-4567
Area: 212 Number: 123-4567


In [55]:
!powershell \
$PhoneRe = [regex]::new('(\d\d\d)-(\d\d\d-\d\d\d\d)') ;\
$Result = $PhoneRe.Match('my number is 212-123-4567') ;\
$Result ;\
$Result.Groups[1].Value ;\
$Result.Groups[2].Value ;\
$Result.Groups[0].Value ;\
$Result.Value ;\
$Full, $Area, $Number = $Result.Groups.Value ;\
Write-Host "Area: $Area Number: $Number"



Groups   : {0, 1, 2}
Success  : True
Name     : 0
Captures : {0}
Index    : 13
Length   : 12
Value    : 212-123-4567

212
123-4567
212-123-4567
212-123-4567
Area: 212 Number: 123-4567




### Find all

In [62]:
ssn = re.compile(r'\d\d\d-\d\d-\d\d\d\d')
ssn.findall('My social is 123-54-3592. Mom\'s is 549-23-1236')

['123-54-3592', '549-23-1236']